### Installations

In [1]:
#choose a minimal T4 GPU execution
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
#check GPU config
!nvidia-smi

Tue Aug 20 06:43:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   40C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import userdata
# Defined in the secrets tab in Google Colab (key icon)
hf_token = userdata.get('HF_TOKEN')

### Functions to compare the models

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [5]:
hf_repo_chocolatine = "jpacifico/Chocolatine-3B-Instruct-DPO-Revised"
hf_repo_phi3 = "microsoft/Phi-3-mini-4k-instruct"

In [6]:
torch.random.manual_seed(0)

def load_model_and_tokenizer(model_name):
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cuda",
        torch_dtype="auto",
        trust_remote_code=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

chocolatine_model, chocolatine_tokenizer = load_model_and_tokenizer(hf_repo_chocolatine)
phi3_model, phi3_tokenizer = load_model_and_tokenizer(hf_repo_phi3)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [7]:
def stream_response(prompt, model, tokenizer, max_new_tokens=500, temperature=0.0):
    messages = [
        {"role": "system", "content": f"Tu es un assistant IA. Ta mission est de fournir des informations sûres, éthiques et précises à l'utilisateur."},
        {"role": "user", "content": prompt},
    ]
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "do_sample": False,
        "streamer": streamer,
    }

    pipe(messages, **generation_args)


In [51]:
user_prompt = """Thomas est en très bonne santé mais il doit aller à l'hôpital tous les jours. Pourquoi ? Donnez plusieurs réponses possibles.", "Peux-tu expliquer en quoi la question ci-dessus est intéressante ?"""

In [52]:
print('**** Phi-3-mini answer :')
stream_response(user_prompt, model=phi3_model, tokenizer=phi3_tokenizer)
print('')
print('**** Chocolatine-3B answer :')
stream_response(user_prompt, model=chocolatine_model, tokenizer=chocolatine_tokenizer)

**** Phi-3-mini answer :
Thomas pourrait aller à l'hôpital tous les jours pour plusieurs raisons qui ne sont pas liées à sa santé. Voici quelques scénarios possibles :

1. Travail : Thomas pourrait être un professionnel de la santé, tel qu'un médecin, un infirmier ou une infirmière, qui doit se rendre à l'hôpital pour effectuer des tâches liées à son emploi.

2. Famille : Il pourrait avoir un membre de sa famille ou un proche qui est hospitalisé et qui lui demande de l'aider ou de passer du temps avec lui.

3. Recherche : Thomas pourrait être un chercheur ou un étudiant en médecine qui doit assister à des séances de formation ou de recherche à l'hôpital.

4. Volontariat : Il pourrait être un bénévole qui aide à l'hôpital, en aidant les patients ou en fournissant des services aux professionnels de la santé.

5. Entraînement : Thomas pourrait être un athlète ou un sportif qui utilise l'hôpital comme lieu d'entraînement, par exemple pour s'entraîner à marcher sur des surfaces lisses ou po